In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0138,  0.0851, -0.1017,  0.0842,  0.0113,  0.2133, -0.0535,  0.0005,
         -0.0345,  0.0528],
        [-0.0519,  0.0524,  0.0995,  0.1972,  0.0709,  0.2025, -0.1841,  0.0155,
         -0.1136,  0.0266]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0995, -0.1797,  0.0238,  0.1680,  0.0258,  0.2447, -0.2060,  0.2799,
          0.1381, -0.1721],
        [ 0.0687, -0.0833, -0.0725,  0.0344, -0.1054,  0.1039, -0.1636,  0.2026,
          0.2355, -0.0640]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
net(X)

tensor([[-0.1581, -0.0553,  0.0518, -0.2939,  0.3417, -0.0626, -0.1697,  0.3204,
          0.2213,  0.0478],
        [-0.0798, -0.0695,  0.0440, -0.1886,  0.2312, -0.1228, -0.0684,  0.1975,
          0.3014,  0.2149]], grad_fn=<AddmmBackward0>)

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.1086, grad_fn=<SumBackward0>)

In [6]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
chimera(X)

tensor(-0.1233, grad_fn=<SumBackward0>)